In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [2]:
children = cg.get_children(864691139914732638)
while children.size:
    print(children.size)
    children = cg.get_children(children, flatten=True)

1
9
9
49
28
238
544
9615
787
2106
39215


In [3]:
data = {
  "sources": [
    [
      "91356497812394262",
      815458.9375,
      884707.6875,
      859720
    ]
  ],
  "sinks": [
    [
      "91356497812401228",
      816143.625,
      884547.3125,
      859720
    ]
  ]
}

In [4]:
from collections import defaultdict
data_dict = {}
for k in ["sources", "sinks"]:
    data_dict[k] = defaultdict(list)
    for node in data[k]:
        node_id = node[0]
        x, y, z = node[1:]
        coordinate = np.array([x, y, z]) / cg.meta._ws_cv.resolution

        atomic_id = cg.get_atomic_id_from_coord(
            coordinate[0],
            coordinate[1],
            coordinate[2],
            parent_id=np.uint64(node_id),
        )

        if atomic_id is None:
            raise ValueError("aha")

        data_dict[k]["id"].append(atomic_id)
        data_dict[k]["coord"].append(coordinate)
data_dict

{'sources': defaultdict(list,
             {'id': [91356497812394262],
              'coord': [Vec(101932.3671875,110588.4609375,21493.0, dtype=float64)]}),
 'sinks': defaultdict(list,
             {'id': [91356497812401228],
              'coord': [Vec(102017.953125,110568.4140625,21493.0, dtype=float64)]})}

In [5]:
from pychunkedgraph.graph.operation import MulticutOperation

op = MulticutOperation(
    cg,
    user_id="test",
    source_ids=data_dict["sources"]["id"],
    sink_ids=data_dict["sinks"]["id"],
    source_coords=data_dict["sources"]["coord"],
    sink_coords=data_dict["sinks"]["coord"],    
    bbox_offset=(240,240,24)
)

op._apply(operation_id="",timestamp=None)

Start get_subgraph
   len(edges) 11926
   4.640780687332153
Start run_multicut
   0.11127448081970215
Start get_parents edges
   7.033348083496094e-05
Cannot find root id 522980506728398854, 10, 2020-02-04 19:22:00.593000+00:00
Cannot find root id 595601050719748099, 10, 2020-02-04 19:22:05.204000+00:00
Cannot find root id 743093938516131845, 12, 2020-02-04 19:22:17.309000+00:00
Cannot find root id 743093938516131846, 12, 2020-02-04 19:22:17.912000+00:00


[864691128455135283, 864691128455135284]

In [8]:
# 91356497812394262
# 91356497812401228

children = cg.get_children(864691128455135283)
while children.size:
    print(children.size)
    children = cg.get_children(children, flatten=True)

1
1
4
6
43
66
270
11373


In [9]:
# 91356497812394262
# 91356497812401228

children = cg.get_children(864691128455135284)
while children.size:
    print(children.size)
    children = cg.get_children(children, flatten=True)

1
1
197
2967
4612
16623
43381
116447
2674542
